In [2]:
import pandas as pd

In [206]:
import requests
import bs4

In [207]:
url = "https://www.imdb.com/list/ls000049962/"
data = requests.get(url)
soup = bs4.BeautifulSoup(data.content, features='html.parser')
links = soup.find_all("a")
link_list=[]
for link in links:
    
    if link.has_attr("href"):
        href = link["href"]
        if href.startswith("/title"):
            link_list.append("https://www.imdb.com/"+href)
           

In [ ]:
title=[]
genre = []
keywords = []
story = []
more_keywords = []

def keywords_func(link):
    result=[]
    data = requests.get(link)
    soup = bs4.BeautifulSoup(data.content, features='html.parser')
    title_div = soup.find_all("div", {"class": "sodatext"})
    for div in title_div:
        result.append(div.a.text)
        
    return result

for link in set(link_list):
    data = requests.get(link)
    soup = bs4.BeautifulSoup(data.content, features='html.parser')
    title_div = soup.find("div", {"class": "title_wrapper"})
    movie_title = title_div.h1.text
    title.append(movie_title)
    
    genre_div = soup.find_all("div", {"class": "see-more inline canwrap"})
    genre_temp = []
    keywords_temp = []
    for div in genre_div:
        
        if div.h4.text == "Genres:":
            for k in div.find_all("a"):
                genre_temp.append(k.text)

        if div.h4.text == "Plot Keywords:":
            for k in div.find_all("a"):
                keywords_temp.append(k.text)
            link = "https://www.imdb.com" + div.find('nobr').a['href']
            more_keywords.append(keywords_func(link))
    
    genre.append(genre_temp)
    keywords.append(keywords_temp)
    
    story_div = soup.find("div", {"class":"inline canwrap"})
    story.append(story_div.p.text)
  
    
    


In [ ]:
import pandas as pd
df1 = pd.DataFrame({
    'title':title,
    'genre':genre,
    'keyword':keywords,
    'story':story,
    'more_keywords':more_keywords
})

In [ ]:
#I have scraped keywords which describe a movie, given by Imdb. I believe this is a good representation of the type of 
#movie, more than genre and storyline. In Keywords, we have been already provided the most important words, which 
#otherwise we have to extract from the storyline. 
#Since all these movies are Imdb top 100, rating would not be useful. Also, movies which are released in the same year
#can be very different, so Year is also not a good measure, thus I have not considered. 

In [14]:
df1 = pd.read_csv('movies4.csv')

In [15]:
df1['story'] = df1['story'].str.replace('\n', '')

In [16]:
df1.head()

,Unnamed: 0,title,genre,keyword,story,more_keywords
0,0,Shichinin no samurai (1954),"[' Action', ' Adventure', ' Drama']","[' samurai', ' 16th century', ' battle', ' jap...","A veteran samurai, who has fallen on hard ...","['samurai', '16th century', 'battle', 'japan',..."
1,1,Cidade de Deus (2002),"[' Crime', ' Drama']","[' photographer', ' slum', ' gang', ' brazil',...","Brazil, 1960s, City of God. The Tender Tri...","['photographer', 'slum', 'gang', 'brazil', 'fa..."
2,2,Casablanca (1942),"[' Drama', ' Romance', ' War']","[' nazi', ' anti nazi', ' casablanca morocco',...","The story of Rick Blaine, a cynical world-...","['nazi', 'anti nazi', 'casablanca morocco', 'f..."
3,3,Se7en (1995),"[' Crime', ' Drama', ' Mystery', ' Thriller']","[' serial killer', ' seven deadly sins', ' det...",A film about two homicide detectives' (Mor...,"['serial killer', 'seven deadly sins', 'detect..."
4,4,Terminator 2: Judgment Day (1991),"[' Action', ' Sci-Fi']","[' time travel', ' liquid metal', ' future', '...",Over 10 years have passed since the first ...,"['time travel', 'liquid metal', 'future', 'seq..."


In [24]:
import nltk
import gensim

In [25]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/shubham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
import string
from nltk.corpus import stopwords

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shubham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
tokenized_text = []

for sent in df1['more_keywords'].values.tolist():
    tokens = word_tokenize(sent)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('','', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    tokenized_text.append(words)
#     print(words)
#     print(word_tokenize(sent))
print(tokenized_text)

[['century', 'favorite', 'sword', 'someone', 'love', 'shaving', 'title', 'situation', 'apprentice', 'relationship', 'blossom', 'love', 'dilemma', 'wife', 'relationship', 'wife', 'death', 'horse', 'sword', 'spear', 'gun', 'danger', 'man', 'woman', 'oneself', 'man', 'woman', 'headed', 'cast', 'sex', 'horse', 'bonding', 'woman', 'killing', 'wife', 'razor', 'black', 'hair', 'alarm', 'friend', 'hair', 'chested', 'male', 'motion', 'scene', 'woman', 'shoulder', 'drawn', 'plow', 'hits', 'daughter', 'paddy', 'elder', 'baby', 'mill', 'riding', 'trunk', 'wood', 'woman', 'man', 'love', 'interest', 'fight', 'hand', 'combat', 'fight', 'fight', 'sword', 'staff', 'someone', 'rifle', 'bow', 'arrow', 'arrow', 'death', 'fire', 'identity', 'ambiguity', 'warrior', 'daughter', 'relationship', 'differences', 'geki', 'arts', 'guy', 'hero', 'hero', 'name'], ['age', 'de', 'janeiro', 'dealer', 'medias', 'res', 'robbery', 'drugs', 'dealer', 'dealer', 'film', 'violence', 'dealing', 'corruption', 'nudity', 'frontal

In [31]:
from gensim.models import Word2Vec 

In [21]:
model = gensim.models.Word2Vec(['air'], min_count=1, size=100, window=1)

In [22]:
words=list(model.wv.vocab)
len(words)

3

In [23]:
model.wv.most_similar('air')

KeyError: "word 'air' not in vocabulary"

In [32]:
#Creating word2vec vectors corresponding to our input keywords.
model2 = gensim.models.Word2Vec(tokenized_text, min_count=1, size=100, window=5)
words=list(model2.wv.vocab)
len(words)

3791

In [36]:
model2.wv.most_similar('brother')

[('death', 0.9856976270675659),
 ('man', 0.9852687120437622),
 ('war', 0.9852622151374817),
 ('someone', 0.9846920967102051),
 ('car', 0.9844560623168945),
 ('head', 0.9843885898590088),
 ('child', 0.9839716553688049),
 ('woman', 0.9839446544647217),
 ('film', 0.9839134216308594),
 ('murder', 0.98366379737854)]

In [ ]:
#Mapping the word vector corresponding to each word in the vocabulary

vector_list = []
for movie in tokenized_text:
    temp = []
    for word in movie:
        temp.append(model2[word])
        print(len(temp))
        
    vector_list.append(temp)

In [44]:
print(len(vector_list))

100


In [131]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

In [ ]:
test_list = np.array(vector_list)
test_list[1]

In [ ]:
#Approach1 : Using the cosine distance between word vectors. I have used keywords from the IMDB website which
#describes a particular movie. 
cosine_list = []

d={}
for i in range(100):
    mean_list=[]
    for j in range(100):
        
        dist_out = pairwise_distances(vector_list[i], vector_list[j], metric="cosine")
        na = np.concatenate(dist_out)
        mean_val = np.mean(na)
        mean_list.append(mean_val)
        print(mean_list)
        

    cosine_list +=[mean_list]
    d[i] = mean_list

#     mean_list.clear()
    
print(cosine_list)
print(d)

In [177]:
len(cosine_list)

100

In [178]:
#let's say Bhavika watched movies no. 35 and 57. We will take the sum of these 2 vectors, and take the maximum cosine distance values. 


summed_list = [sum(i) for i in zip(cosine_list[34], cosine_list[56])] 

In [184]:
number_list = list(range(100))
print(sorted(zip(summed_list, number_list), reverse=True)[:3])

[(1.034047245979309, 72), (0.955760657787323, 83), (0.9412471950054169, 65)]


In [205]:
movie_list = df1['title'].values.tolist()
print("movies watched:", movie_list[34],movie_list[56])
print("recommended movies:", movie_list[72],movie_list[83],movie_list[65])

movies watched: Rear Window (1954)  Unforgiven (1992) 
recommended movies: All About Eve (1950)  Das Boot (1981)  Ladri di biciclette (1948) 


In [ ]:
#Approach2: Autoencoder. Autoencoder provided a compact representaion of the input keeping the most important features.
#I want to use this dense representaion for distance measures.

In [46]:
import tensorflow as tf

In [47]:
tf.__version__

'2.0.0'

In [49]:
import numpy as np

In [199]:
input_list = [np.mean(k,axis=0) for k in vector_list]

In [ ]:
input_list[0]

In [201]:
input_list_np = np.array(input_list)
input_list_np.shape

(100, 100)

In [117]:
class AutoEncoder(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.input_layer = tf.keras.layers.Dense(100, activation='relu')
        self.encoder_hidden_layer1 = tf.keras.layers.Dense(64, activation='sigmoid')
        self.encoder_hidden_layer2 = tf.keras.layers.Dense(32, activation="sigmoid")
        self.decoder_hidden_layer1 = tf.keras.layers.Dense(64, activation='relu')
        self.output_layer = tf.keras.layers.Dense(100, activation='sigmoid')
        
        
    def call(self, inputs):
        input_ = self.input_layer(inputs)
        hidden1 = self.encoder_hidden_layer1(input_)
        hidden2 = self.encoder_hidden_layer2(hidden1)
        decoder_hidden = self.decoder_hidden_layer1(hidden2)
        output = self.output_layer(decoder_hidden)
        
        return output, hidden2
    

In [118]:
opt = tf.optimizers.Adam(learning_rate=learning_rate)

def loss(model, original):
    model_output, hidden_output = model(original)
    reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(model_output, original)))
    return reconstruction_error
  
def train(loss, model, opt, original):
    
    with tf.GradientTape() as tape:
        gradients = tape.gradient(loss(model, original), model.trainable_variables)
        gradient_variables = zip(gradients, model.trainable_variables)
        opt.apply_gradients(gradient_variables)

In [113]:
training_dataset = tf.data.Dataset.from_tensor_slices(input_list_np)
training_dataset = training_dataset.batch(1)

In [119]:
np.random.seed(1)
tf.random.set_seed(1)

epochs = 10
learning_rate = 1e-3

autoencoder = AutoEncoder()
for epoch in range(epochs):
    
    for batch_features in (training_dataset):
        train(loss, autoencoder, opt, batch_features)
        loss_values = loss(autoencoder, batch_features)
        print(loss_values)

tf.Tensor(0.25779602, shape=(), dtype=float32)
tf.Tensor(0.25459734, shape=(), dtype=float32)
tf.Tensor(0.2514452, shape=(), dtype=float32)
tf.Tensor(0.2485174, shape=(), dtype=float32)
tf.Tensor(0.2456447, shape=(), dtype=float32)
tf.Tensor(0.24280673, shape=(), dtype=float32)
tf.Tensor(0.23977579, shape=(), dtype=float32)
tf.Tensor(0.23708767, shape=(), dtype=float32)
tf.Tensor(0.23416324, shape=(), dtype=float32)
tf.Tensor(0.23133825, shape=(), dtype=float32)
tf.Tensor(0.22822315, shape=(), dtype=float32)
tf.Tensor(0.22532035, shape=(), dtype=float32)
tf.Tensor(0.22178549, shape=(), dtype=float32)
tf.Tensor(0.21868467, shape=(), dtype=float32)
tf.Tensor(0.21517831, shape=(), dtype=float32)
tf.Tensor(0.2115271, shape=(), dtype=float32)
tf.Tensor(0.2077457, shape=(), dtype=float32)
tf.Tensor(0.20382069, shape=(), dtype=float32)
tf.Tensor(0.19981566, shape=(), dtype=float32)
tf.Tensor(0.19579698, shape=(), dtype=float32)
tf.Tensor(0.19138835, shape=(), dtype=float32)
tf.Tensor(0.186943

In [ ]:
#taking output from the hidden layer of the autoencoder.
autoencoder_features = hidden_output(autoencoder, input_list_np)
autoencoder_features

In [189]:
#converting to numpy
autoencoder_features_numpy = autoencoder_features.numpy()
autoencoder_features_numpy.shape

(100, 32)

In [ ]:
cosine_list2 = []

d2={}
for i in range(100):
    print("cosine beginning", cosine_list2)
    mean_list2=[]
    for j in range(100):
        
        dist_out = cosine(autoencoder_features_numpy[i], autoencoder_features_numpy[j])
#         na = np.concatenate(dist_out)
        mean_val = np.mean(dist_out)
        mean_list2.append(mean_val)
        print(mean_list)
        
    print("here",mean_list2)
    cosine_list2 +=[mean_list2]
    d[i] = mean_list2
    print("sub-final", cosine_list2)
#     mean_list.clear()
    
print(cosine_list2)
print(d2)

In [194]:
len(cosine_list2)

100

In [195]:
summed_list = [sum(i) for i in zip(cosine_list2[34], cosine_list2[56])] 
number_list = list(range(100))
print(sorted(zip(summed_list, number_list), reverse=True)[:3])

[(2.467632293701172e-05, 83), (1.9431114196777344e-05, 36), (1.627206802368164e-05, 72)]


In [211]:
print("movies watched: ",movie_list[34],movie_list[56])
print("movies recommended: ",movie_list[83],movie_list[36],movie_list[72])

movies watched:  Rear Window (1954)  Unforgiven (1992) 
movies recommended:  Das Boot (1981)  The Shawshank Redemption (1994)  All About Eve (1950) 


In [ ]:
#Further Imporvements: Here I have used only Keywords for the input. I could have also used the storyline, and passed it
#in the autoencoder as an input to get a combined representation.